In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# To list files in the directory where you saved the model
import os
os.listdir("/content/drive/MyDrive/Apple_dataset")  # Look for your model file in the MyDrive folder

['final_model.keras',
 'test',
 'train',
 'CheckPoints',
 'final_apple_model.keras']

In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained Apple disease model
model_path = "/content/drive/MyDrive/Apple_dataset/final_apple_model.keras"
model = load_model(model_path)

# Provide the full path to your test image
test_image_path = "/content/drive/MyDrive/Apple_dataset/test/black_rot/02168189-aa75-4284-a7f0-8ca5901ea783___JR_FrgE.S 2948_90deg.JPG"

# ✅ Define class labels used during training
class_labels = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy'
]

# 🔍 Known actual class of this test image (manually specify or read from filename)
actual_class_name = 'Apple___Black_rot'  # ← Update this as per your test image

# Read and preprocess the image
image = cv2.imread(test_image_path)
if image is None:
    print("❌ Error: Image not found at the specified path.")
else:
    image = cv2.resize(image, (256, 256))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    # Make a prediction
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    predicted_class_name = class_labels[predicted_label]

    # ✅ Print both actual and predicted class
    print(f"✅ Actual Class:    {actual_class_name}")
    print(f"✅ Predicted Class: {predicted_class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
✅ Actual Class:    Apple___Black_rot
✅ Predicted Class: Apple___Black_rot


In [5]:
code = '''
import streamlit as st
import numpy as np
import tensorflow as tf
import cv2
from keras.preprocessing.image import img_to_array
from PIL import Image
import base64
from io import BytesIO

# Load the trained model
model_path = "/content/drive/MyDrive/Apple_dataset/final_apple_model.keras"
model = tf.keras.models.load_model(model_path)

class_labels = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy'
]

care_recommendations = {
    'Apple___Apple_scab': [
        "Prune infected leaves and twigs.",
        "Apply fungicide in early spring.",
        "Avoid overhead irrigation."
    ],
    'Apple___Black_rot': [
        "Remove and destroy infected fruit and leaves.",
        "Use fungicide sprays during the growing season.",
        "Clean up plant debris regularly."
    ],
    'Apple___Cedar_apple_rust': [
        "Remove nearby cedar trees if possible.",
        "Use resistant apple varieties.",
        "Apply fungicides during early spring."
    ],
    'Apple___healthy': [
        "Maintain proper spacing for airflow.",
        "Regularly inspect leaves for early signs of disease.",
        "Ensure proper fertilization and watering schedule."
    ]
}

def preprocess_image(image):
    image = cv2.resize(image, (256, 256))
    image = img_to_array(image)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

st.set_page_config(page_title="Apple Leaf Disease Detector", page_icon="🌿", layout="centered")

st.markdown("""<style>
.result-card {
    padding: 1rem;
    border-radius: 0.5rem;
    margin-top: 1rem;
}
.result-card.healthy {
    background-color: #e0f7e9;
    border: 1px solid #34a853;
}
.result-card.diseased {
    background-color: #ffe0e0;
    border: 1px solid #ea4335;
}
.confidence-meter {
    background-color: #f0f0f0;
    border-radius: 20px;
    height: 20px;
    margin-top: 10px;
}
.confidence-fill {
    background-color: #34a853;
    height: 100%;
    border-radius: 20px;
}
</style>""", unsafe_allow_html=True)

st.markdown("""
    <div class="header">
        <h1><b>🌿 Botanical Care:</b></h1>
        <h2>A Plant Disease Detection System</h2>
        <p>Upload a leaf image to check for common diseases</p>
    </div>
""", unsafe_allow_html=True)

uploaded_file = st.file_uploader("Choose an apple leaf image...", type=["jpg", "png", "jpeg"], label_visibility="collapsed")

if uploaded_file is not None:
    with st.spinner('Analyzing your leaf...'):
        # Load image and show original size using HTML
        pil_image = Image.open(uploaded_file)
        buffered = BytesIO()
        pil_image.save(buffered, format="PNG")
        img_b64 = base64.b64encode(buffered.getvalue()).decode()
        st.markdown(f"""
            <div style="text-align:center; margin-bottom: 10px;">
                <img src="data:image/png;base64,{img_b64}" style="max-width:100%; height:auto;"/>
            </div>
        """, unsafe_allow_html=True)

        # Prepare for prediction
        image_np = np.array(pil_image.convert('RGB'))
        image_cv2 = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        processed_image = preprocess_image(image_cv2)
        prediction = model.predict(processed_image)
        predicted_class = class_labels[np.argmax(prediction)]
        confidence = np.max(prediction) * 100

        st.markdown(f'<div class="result-card {"healthy" if "healthy" in predicted_class else "diseased"}">', unsafe_allow_html=True)

        threshold = 60
        if confidence < threshold:
            st.warning("Unable to confidently detect disease. Try uploading a clearer apple leaf image.")
        else:
            if "healthy" in predicted_class:
                st.success("🍃 The apple leaf is healthy.")
            else:
                st.error(f"🚨 Disease Detected: {predicted_class.replace('___', ' ').title()}")

            st.write(f"**Confidence:** {confidence:.1f}%")
            st.markdown(f"""
                <div class="confidence-meter">
                    <div class="confidence-fill" style="width:{confidence}%"></div>
                </div>
            """, unsafe_allow_html=True)

            with st.expander("📌 Care Recommendations"):
                for tip in care_recommendations[predicted_class]:
                    st.write(f"- {tip}")

        st.markdown('</div>', unsafe_allow_html=True)

st.markdown("""
    <div style="text-align: center; color: #666; margin-top: 2rem;">
        <small>For accurate diagnosis, consult an agricultural expert.</small>
    </div>
""", unsafe_allow_html=True)

'''

# ✅ Save it to app.py
with open("/content/drive/MyDrive/Apple_dataset/app.py", "w") as f:
    f.write(code)

print("✅ app.py saved successfully to Google Drive.")


✅ app.py saved successfully to Google Drive.


In [6]:
!pip install streamlit pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install streamlit -q
!pip install pyngrok

In [8]:
!streamlit version

Streamlit, version 1.46.1


In [9]:
!ngrok authtoken 2v9qExBrDuG7JJbV2OWF6W4ruKZ_5GJgSkP7crEPCrHck2Ywj

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
!pkill -f streamlit
!pkill -f ngrok

In [11]:
!streamlit run /content/drive/MyDrive/Apple_dataset/app.py &>/dev/null &

In [12]:
!mkdir -p ~/.streamlit/logs
!streamlit run app.py --server.port 8501 > ~/.streamlit/logs/streamlit.log 2>&1 &

In [13]:
!lsof -i :8501

COMMAND    PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
streamlit 4073 root    6u  IPv4 125826      0t0  TCP *:8501 (LISTEN)
streamlit 4073 root    7u  IPv6 125827      0t0  TCP *:8501 (LISTEN)


In [14]:
from pyngrok import ngrok

# Restart Ngrok on port 8501
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://2d714386aa84.ngrok-free.app" -> "http://localhost:8501"
